In [17]:
import gc
import joblib

import sklearn as sk
import pandas as pd
import numpy as np
import pathlib as pl

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error as mse

from sklearn import datasets
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
#from sklearn.ensemble import RandomForestRegressor as RFR
#from xgboost import XGBRegressor as XGBR
#from lightgbm import LGBMRegressor as LGBM

data_dir = pl.Path("D:/Data/open-problems-multimodal/processed")

In [3]:
train_input = pd.read_csv(data_dir / "internal_train_train_cite_inputs.csv", index_col=0)
train_target = pd.read_csv(data_dir / "internal_train_train_cite_targets.csv", index_col=0)

In [4]:
print(train_input)
print(train_target)

              ENSG00000121410_A1BG  ENSG00000268895_A1BG-AS1  \
cell_id                                                        
45006fe3e4c8                   0.0                       0.0   
d02759a80ba2                   0.0                       0.0   
c016c6b0efa5                   0.0                       0.0   
ba7f733a4f75                   0.0                       0.0   
fbcf2443ffb2                   0.0                       0.0   
...                            ...                       ...   
0169f964147e                   0.0                       0.0   
7203b2ace768                   0.0                       0.0   
834449e1a23d                   0.0                       0.0   
769790e1b39a                   0.0                       0.0   
e7012ac38766                   0.0                       0.0   

              ENSG00000175899_A2M  ENSG00000245105_A2M-AS1  \
cell_id                                                      
45006fe3e4c8                  0.0          

In [5]:
print(train_input.shape)
print(train_target.shape)

(42843, 22050)
(42843, 140)


In [6]:
# Run PCA
p = 500

train_input_sd = StandardScaler().fit_transform(train_input)
train_input_pca = PCA(n_components=p)
train_input_pca.fit(train_input_sd)
X = pd.DataFrame(train_input_pca.transform(train_input_sd))


In [13]:
y = train_target

In [7]:
X.to_csv(data_dir/"internal_train_train_cite_inputs_pca.csv")

In [10]:
# joblib.dump(train_input_pca, "train_input_pca.m")

['train_input_pca.m']

In [11]:
# train_input_pca2 = joblib.load("train_input_pca.m")

In [20]:
print(X.shape)
print(y.shape)

(42843, 500)
(42843, 140)


In [14]:
# train test split
# test_size = 0.2
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(34274, 500) (8569, 500) (34274, 140) (8569, 140)


In [15]:
# fit the model
a = 0.1
# model = Lasso(alpha=a)
# model = Ridge(alpha=a)
# model = RFR(n_estimators=10, max_depth=5)
# model = XGBR(max_depth=5, learning_rate=0.1, n_estimators=20)
# model = LGBM(objective='regression',num_leaves=31,learning_rate=0.05,n_estimators=20))
# model.fit(X_train, y_train)


In [18]:
# Lasso
model = Lasso(alpha=a)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
test_mse = mse(y_test, y_pred)
test_pear = pearsonr(y_test.to_numpy().flatten(), y_pred.flatten())
print("MSE of the test set:", test_mse)
print("Pearson of the test set:", test_pear[0])

MSE of the test set: 2.5299839839463654
Pearson of the test set: 0.8889734341295122


In [36]:
param_grid = [{'alpha': np.arange(0.02,0.1,0.02)}]
 
model = Lasso()
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
 
grid_search.fit(X, y)

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid=[{'alpha': array([0.02, 0.04, 0.06, 0.08])}],
             scoring='neg_mean_squared_error')

In [37]:
grid_search.cv_results_

{'mean_fit_time': array([15.21143022, 13.73326783, 12.8338109 , 12.49468341]),
 'std_fit_time': array([1.37646153, 0.78187078, 0.57109873, 0.37686674]),
 'mean_score_time': array([0.04541097, 0.04160881, 0.04013386, 0.03800263]),
 'std_score_time': array([0.00467479, 0.00215438, 0.00240104, 0.00208989]),
 'param_alpha': masked_array(data=[0.02, 0.04, 0.06, 0.08],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'alpha': 0.02},
  {'alpha': 0.04},
  {'alpha': 0.06},
  {'alpha': 0.08}],
 'split0_test_score': array([-2.51451555, -2.50378059, -2.4976348 , -2.49288784]),
 'split1_test_score': array([-2.90676943, -2.92706062, -2.94810158, -2.96781905]),
 'split2_test_score': array([-3.38268348, -3.40973928, -3.43609102, -3.46042356]),
 'split3_test_score': array([-2.20863744, -2.21415563, -2.22253823, -2.23171795]),
 'split4_test_score': array([-2.37285687, -2.37754626, -2.38600653, -2.39454779]),
 'mean_test_score': array([-2.6

In [31]:
best_model = grid_search.best_estimator_
joblib.dump(best_model, "lasso.m")

['lasso.m']

In [32]:
test_input = pd.read_csv(data_dir / "internal_holdout_train_cite_inputs.csv", index_col=0)
test_target = pd.read_csv(data_dir / "internal_holdout_train_cite_targets.csv", index_col=0)

In [1]:
X_test = train_input_pca.transform(test_input)
y_test = test_target
y_pred = best_model.predict(X_test)

train_pear = pearsonr(y.to_numpy().flatten(), best_model.predict(X))
test_pear = pearsonr(y_test.to_numpy().flatten(), y_pred.flatten())
print("Pearson of the test set:", test_pear[0])

NameError: name 'train_input_pca' is not defined

In [150]:
# Ridge
model = Ridge(alpha=a)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
test_mse = mse(y_test, y_pred)
test_pear = pearsonr(y_test.to_numpy().flatten(), y_pred.flatten())
print("MSE of the test set:", test_mse)
print("Pearson of the test set:", test_pear[0])

MSE of the test set: 2.963454733984181
Pearson of the test set: 0.8636707792964627


In [151]:
# Random Forest Regressor
model = RFR(n_estimators=10, max_depth=5)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
test_mse = mse(y_test, y_pred)
test_pear = pearsonr(y_test.to_numpy().flatten(), y_pred.flatten())
print("MSE of the test set:", test_mse)
print("Pearson of the test set:", test_pear[0])

MSE of the test set: 3.2406228916576416
Pearson of the test set: 0.8496626252995269


In [186]:
# XGBoost Regressor
model = XGBR(max_depth=3, learning_rate=0.1, n_estimators=10)
y_preds = np.array([])
for i in range(y.shape[1]):
    model.fit(X_train, y_train.iloc[:,i])
    y_pred = model.predict(X_test)
    y_preds = np.concatenate((y_preds, y_pred), axis=0)

test_mse = mse(y_test.to_numpy().flatten("F"), y_preds)
test_pear = pearsonr(y_test.to_numpy().flatten("F"), y_preds)
print("MSE of the test set:", test_mse)
print("Pearson of the test set:", test_pear[0])

MSE of the test set: 4.487843535841501
Pearson of the test set: 0.8537708949008104


In [184]:
# LightGBM
model = LGBM(objective='regression',num_leaves=5,learning_rate=0.1,n_estimators=10)
y_preds = np.array([])
for i in range(y.shape[1]):
    model.fit(X_train, y_train.iloc[:,i])
    y_pred = model.predict(X_test)
    y_preds = np.concatenate((y_preds, y_pred), axis=0)

test_mse = mse(y_test.to_numpy().flatten("F"), y_preds)
test_pear = pearsonr(y_test.to_numpy().flatten("F"), y_preds)
print("MSE of the test set:", test_mse)
print("Pearson of the test set:", test_pear[0])

MSE of the test set: 3.4983781792748796
Pearson of the test set: 0.8375270304196588
